In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm

# 
import numpy as np
import os
from tqdm import trange
import parmap
import glob
from sklearn.decomposition import PCA
#import umap
import seaborn as sns
import pandas as pd

import pickle
# 
from tqdm import tqdm

import sleap


import sklearn.experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 
from scipy.io import loadmat
import scipy
import scipy.ndimage
# 

import numba
from numba import jit


from Imputation import Impute, CentreBody

from Imputation import *

Autosaving every 180 seconds


In [2]:
##################################################   
############# GENERATE EGOCENTRIC DATA ###########   
##################################################
cb = CentreBody()
cb.parallel = True
cb.root_dir = '/media/cat/1TB/dan/cohort1/slp/'
#cb.root_dir = '/media/cat/256GB/dan/slp'

#
#cb.process_slp()

#
cb.get_fnames()

# median filter data
cb.filter_data()

# 
cb.reject_outliers()



  ... median filtering ...


100%|██████████| 23/23 [00:00<00:00, 296827.67it/s]

  ... rejecting outliers....



100%|██████████| 23/23 [00:00<00:00, 22968.81it/s]


In [56]:
##################################################   
####### GENERATE PAIR-WISE CENTRE DATASETS #######   
################################################## 

# 
cb.parallel = True

# for each recording centre/align every pair of data
#     TODO: make a single file otherwise generating 15 files per recording
#      TODO : OR MAKE A SINGLE STACK OF DATA
cb.centre_and_align_all_pairs()


100%|██████████| 6/6 [03:29<00:00, 34.95s/it]

 DONE Generating 2-point ground truth datasets for imputation


In [6]:
#################################################        
#### TRAIN PAIRWISE MODELS MISSING FEATURES #####
#################################################                       

# pass cb object into Impute object
I = Impute(cb)
I.parallel = False
I.n_cores = 4

# Select best model; 
#  ["BayesianRidge",          0
#   "DecisionTreeRegressor",  1 
#   "ExtraTreesRegressor",    2  <--- best performing when tested on female data; but large data, 2.5GB /model
#   "KNeighborsRegressor"]    3  <--- 2nd best
#  VAE also evaluated; but not coded
models = [0]
for animal_id in I.animal_ids:
    I.animal_id = animal_id
    for model in models:
        I.model_type = model

        # this will generate 4 animals x 15 pairwise models = 60 models
        I.generate_imputation_models_all_pairs()



   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:

 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup

 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup1:                               (132861, 6, 2)
 pup2:                               (176982, 6, 2)
   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (218641, 6, 2)
 male:                               (94647, 6, 2)
 pup

In [ ]:
# #######################################################    
# #### LOAD GT DATA AND PREDICT USING SPECIFIC MODEL ####
# #######################################################
# # select a particular animal
# I = Impute(cb)
# I.parallel = True
# I.n_cores = 4
# I.model_type = 0 

# #
# I.animals_selected = [0]
# I.animal_id = 0
# I.generate_random_drops = True     # flag used to indicate 

# # this generates random drops file + evaluates how the trained model does
# #I.predict_imputation_ground_truth_all_pairs()

# # plot violion plots of errors
# #plot_errors(I)

# print ("DONE")

In [115]:
# #########################################################    
# #### LOAD REAL DATA AND PREDICT USING SPECIFIC MODEL ####
# #########################################################
# # select a particular animal
# I = Impute(cb)
# I.parallel = False
# I.n_cores = 8
# I.model_type = 0

# # 
# I.animals_selected = [0]
# I.generate_random_drops = False     # flag used to indicate 

# # 
# # I.predict_novel_data()




In [9]:
################################
#### MAKE ALL_DATA DATASETS ####
################################

# TODO: EMBED THIS INTO ANOTHER FUNCTION EARLY IN PROCESSING
#      TODO: probably better to process each hour of data as stand alone - don't want concatenation artifacts etc.

def make_this_into_a_function()
    f1 = None
    f2 = None
    feats = load_processed_data_stand_alone(f1,f2,I.March16_file_order, I.root_dir, I.animal_ids, data_type=1, remove_nans=False)
    print (np.vstack(feats[0]).shape)
    for k in range(I.animal_ids.shape[0]):
        fname_out = I.root_dir+'/animalID_'+str(k)+'_alldata.npy'
        if os.path.exists(fname_out)==False:
            np.save(fname_out, np.vstack(feats[k]))



   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (2069710, 14, 2)
 male:                               (2069710, 14, 2)
 pup1:                               (2069710, 14, 2)
 pup2:                               (2069710, 14, 2)
(2069710, 14, 2)


In [116]:
###############################################
#### FIND FRAMES WHICH HAVE F1/F2 ANCHORS #####
###############################################

# parse the data and resave chunks containing centre/anchor point pairs
for animal_id in I.animal_ids:
    I.animal_id = animal_id
    for f1 in range(I.feature_ids.shape[0]):
        for f2 in range(f1+1, I.feature_ids.shape[0],1):
            break_features_pairs(f1,f2, I)



(2069710, 14, 2)
INFO:numba.core.transforms:finding looplift candidates


/home/cat/code/gerbil/movement_analysis/Imputation.py:609: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function parse_data_for_anchors failed at nopython mode lowering due to: non-numeric type in Num

File "Imputation.py", line 615:
def parse_data_for_anchors(f1, f2, feats):
    <source elided>
        # count # of non nan entries
        idx = np.where(np.isnan(feats[k,:,0])==False)[0]
        ^

During: lowering "$24.17 = arrayexpr(expr=(<built-in function eq>, [(<ufunc 'isnan'>, [Var($24.14, Imputation.py:615)]), const(bool, False)]), ty=array(bool, 1d, C))" at /home/cat/code/gerbil/movement_analysis/Imputation.py (615)
  @numba.jit()
/home/cat/code/gerbil/movement_analysis/Imputation.py:609: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "parse_data_for_anchors" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "Imputatio

(628094,)
(2069710, 14, 2)
(518377,)
(2069710, 14, 2)
(409718,)
(2069710, 14, 2)
(246257,)
(2069710, 14, 2)
(230403,)
(2069710, 14, 2)
(565952,)
(2069710, 14, 2)
(453972,)
(2069710, 14, 2)
(274308,)
(2069710, 14, 2)
(248357,)
(2069710, 14, 2)
(441885,)
(2069710, 14, 2)
(260741,)
(2069710, 14, 2)
(237260,)
(2069710, 14, 2)
(269522,)
(2069710, 14, 2)
(244677,)
(2069710, 14, 2)
(264280,)
(2069710, 14, 2)
(363199,)
(2069710, 14, 2)
(203186,)
(2069710, 14, 2)
(151170,)
(2069710, 14, 2)
(118263,)
(2069710, 14, 2)
(120073,)
(2069710, 14, 2)
(252295,)
(2069710, 14, 2)
(174017,)
(2069710, 14, 2)
(142856,)
(2069710, 14, 2)
(153324,)
(2069710, 14, 2)
(160547,)
(2069710, 14, 2)
(116635,)
(2069710, 14, 2)
(117849,)
(2069710, 14, 2)
(128634,)
(2069710, 14, 2)
(123964,)
(2069710, 14, 2)
(178110,)
(2069710, 14, 2)
(461638,)
(2069710, 14, 2)
(302560,)
(2069710, 14, 2)
(216636,)
(2069710, 14, 2)
(146693,)
(2069710, 14, 2)
(176257,)
(2069710, 14, 2)
(313498,)
(2069710, 14, 2)
(223687,)
(2069710, 14, 2)
(

In [117]:
#########################################
#### CENTRE AND ROTATE F1/F2 FRAMES #####
#########################################
''' This step takes all f1/f2 eligible frames
'''
# 
for animal_id in I.animal_ids:
    I.animal_id = animal_id
    I.parallel = True
    centre_rotate_features_pairs(I)
            

# 

100%|██████████| 427591/427591 [00:24<00:00, 17604.97it/s]


In [118]:
#########################################################
#### IMPUTE MISSING DATA ON THE ROTATED FRAMES ONLY #####
#########################################################

#  This function loads all data (including nans), + an f1/f2 model, and imputes 
#  missing data for frames that have at least 3 features including the f1/f2 points
# 
I = Impute(cb)
I.parallel = True
I.n_cores = 4
I.model_type = 0 

# for animal_id in I.animal_ids:
#     I.animal_id = animal_id
I.animal_ids = np.arange(4)
I.parallel = False
impute_real_data_stand_alone(I)





   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (2069710, 6, 2)
 male:                               (2069710, 6, 2)
 pup1:                               (2069710, 6, 2)
 pup2:                               (2069710, 6, 2)
0 1  frame_ids with min 3 present features in animal:  0  is:  (628094,)


100%|██████████| 628094/628094 [00:06<00:00, 99136.96it/s] 




0 1  frame_ids with min 3 present features in animal:  1  is:  (363199,)


100%|██████████| 363199/363199 [00:03<00:00, 105458.08it/s]




0 1  frame_ids with min 3 present features in animal:  2  is:  (461638,)


...



4 5  frame_ids with min 3 present features in animal:  2  is:  (154460,)


100%|██████████| 154460/154460 [00:01<00:00, 103522.88it/s]


  0%|          | 0/279656 [00:00<?, ?it/s]

4 5  frame_ids with min 3 present features in animal:  3  is:  (279656,)


100%|██████████| 279656/279656 [00:02<00:00, 100786.07it/s]


In [144]:
################################################################
#### REPLACE MISSING DATA WITH MODELS WITH NEARBY FEATURES #####
################################################################

I = Impute(cb)
I.parallel = True
I.n_cores = 4
I.model_type = 0 

I.animal_ids = np.arange(4)
I.parallel = False

# figure out which model is most optimal for each missing feature
# e.g. missing feature 0 ideal model is 1/2 
best_models = [              # missing feature index
[[1,2],[1,3]],               # 0
[[0,2],[0,3],[2,3]],         # 1
[[1,3],[1,4],[0,3],[0,1]],         # 2
[[2,4],[2,5],[1,4],[1,2]],   # 3
[[3,5],[2,5],[2,3]],         # 4
[[3,4],[2,4]]                # 5
]
print (len(best_models))

#########################################################
animal_ids = np.arange(4)
for animal_id in animal_ids:

    #####################################################
    # load all imputations for that animal_id
    feats_imputed_array = []
    feats_frames_array = []
    for f1 in range(6):
        feats_imputed_array.append([])
        feats_frames_array.append([])
        for f2 in range(6):
            feats_imputed_array[f1].append([])
            feats_frames_array[f1].append([])

    # fill the lists with imputed data
    for f1 in trange(6):
        for f2 in range(f1+1, 6, 1):

            fname_imputed = os.path.join(I.root_dir, 'centre_rotated_animalID_'+str(animal_id)+
                                 "_"+str(f1)+"_"+str(f2)+"_imputed_original_locations.npz")

            data = np.load(fname_imputed)
            temp1 = data['imputed_translated']
            temp2 = data['imputed_frames']

            feats_imputed_array[f1][f2]= temp1
            feats_frames_array[f1][f2]= temp2


    #####################################################
    # load raw unrotated data
    feats = np.load(os.path.join(I.root_dir,'animalID_'+str(animal_id)+"_alldata.npy"))
    feats_a = feats[:,I.feature_ids]
    print ("feats_a: ", feats_a.shape)

    # initialize imputed array 
    feats_imputed = np.zeros(feats_a.shape,'float32')+np.nan
    print ("feats_imputed: ", feats_imputed.shape)

    # loop over all frames
    fname_out = os.path.join(I.root_dir, 'animalID_'+str(animal_id)+"_alldata_imputed.npy")

    #if os.path.exists(fname_out)==False:
    verbose=False
    if True:

        all_feats = np.arange(6)
        for f in trange(feats_a.shape[0]):
            feats_orig = feats_a[f]
            feats_imputed[f] = feats_orig

            # grab indexes for nans 
            idxnan = np.where(np.isnan(feats_orig[:,0]))[0]
            idx = np.delete(all_feats, idxnan)    
            #print ("frame: ", f, " idx: ", idxnan)

            # if less than 2 anchors skip frame
            if idx.shape[0]<=1:
                continue
            
            # loop over all missing pieces; replace all nans with data from best model regardless of how many nans are present
            for id_ in idxnan:
                if verbose:
                    print ("frame: ", f, "missing feature id: ", id_, "  available features: ", idx)

                # search for the best model in order of best fit (as listed above)
                models1 = best_models[id_]
                for mod in models1:

                    # if model is found (i.e. the anchor points are availble): load files and replace data
                    if np.intersect1d(mod,idx).shape[0]==2:
                        #print ("MODEL FOUND: ", mod)

                        #################################
                        frames = feats_frames_array[mod[0]][mod[1]]

                        # some of the models were not used to impute 
                        try:
                            idx_local = np.where(frames==f)[0][0]
                        except:
                            continue

                        temp = feats_imputed_array[mod[0]][mod[1]]
                        temp = temp[idx_local][id_] 

                        feats_imputed[f,id_] = temp
                        
                        if verbose:
                            print ("Grabbing model ", mod, " imputation results at location: ", id_, " temp: ", temp)

                        # do not look for other models to impute at this location;
                        break
            if verbose:
                print ('')
        np.save(fname_out, feats_imputed)


 17%|█▋        | 1/6 [00:00<00:00,  9.76it/s]

6


  0%|          | 4700/2069710 [00:00<00:43, 46973.50it/s]

feats_a:  (2069710, 6, 2)
feats_imputed:  (2069710, 6, 2)


  0%|          | 5148/2069710 [00:00<00:40, 51477.10it/s]

...

  0%|          | 3497/2069710 [00:00<00:59, 34886.77it/s]

feats_a:  (2069710, 6, 2)
feats_imputed:  (2069710, 6, 2)


  0%|          | 898/2069710 [00:00<03:50, 8978.08it/s]

feats_a:  (2069710, 6, 2)
feats_imputed:  (2069710, 6, 2)


100%|██████████| 2069710/2069710 [03:04<00:00, 11218.43it/s]


In [199]:
#
animal_ids = np.arange(4)
for animal_id in animal_ids:
    plt.subplot(2,2,animal_id+1)
    feature_ids = np.array([0,5,6,7,8,9])
    d = np.load('/media/cat/1TB/dan/cohort1/slp/animalID_'+str(animal_id)+'_alldata.npy')[:,feature_ids]
    d_imp = np.load('/media/cat/1TB/dan/cohort1/slp/animalID_'+str(animal_id)+'_alldata_imputed.npy')

    n_feats = []
    n_feats_imp = []
    for k in trange(d.shape[0]):
        idx = np.array(np.where(np.isnan(d[k,:,0])==False)[0])
        idx2 = np.array(np.where(np.isnan(d_imp[k,:,0])==False)[0])

        #print (k, idx, idx.shape)
        n_feats.append(idx.shape[0])
        n_feats_imp.append(idx2.shape[0])

    n_feats = np.array(n_feats)
    print (n_feats.shape)
    n_feats_imp = np.array(n_feats_imp)

    width = 1
    bins = np.arange(-0.5,7,width)

    y = np.histogram(n_feats,bins=bins)
    y2 = np.histogram(n_feats_imp, bins=bins)

    plt.plot(y[1][1:]-0.5,y[0],c='black', label = 'Sleap')
    plt.plot(y2[1][1:]-0.5,y2[0],c='blue', label = 'Imputation')

    cumsum = np.cumsum(y[0][::-1])
    plt.plot(y[1][1:][::-1]-0.5,cumsum, c= 'red', label = "Sleap - cumulative sum",
            linewidth=5)
    cumsum = np.cumsum(y2[0][::-1])
    plt.plot(y[1][1:][::-1]-0.5,cumsum, '--', c= 'red', label = "Imputation - cumulative sum",
            linewidth=5)
    
    #plt.semilogy()
    plt.xlim(0.5,6.5)
    plt.ylim(0,750000)
    plt.legend(fontsize=10)
    plt.title("animal id: "+ str(animal_id))
    plt.ylabel("# frames")
    plt.xlabel("# of features in frame")
    plt.plot([3,3],[0,750000],'--',c='black',linewidth=3,alpha=.7)

plt.show()

100%|██████████| 2069710/2069710 [00:19<00:00, 105671.41it/s]


(2069710,)


100%|██████████| 2069710/2069710 [00:19<00:00, 106272.33it/s]


(2069710,)


100%|██████████| 2069710/2069710 [00:19<00:00, 105952.57it/s]


(2069710,)


100%|██████████| 2069710/2069710 [00:19<00:00, 108632.53it/s]


(2069710,)


In [193]:
print(y[0])
print (np.cumsum(y[0][::-1]))

[1243944   86107  160024  146467  182678   31808  218682]
[ 218682  250490  433168  579635  739659  825766 2069710]


In [187]:
n_feats = np.array(n_feats)
print (n_feats.shape)
n_feats_imp = np.array(n_feats_imp)

width = 1
bins = np.arange(0.5,7,width)

y = np.histogram(n_feats,bins=bins)
y2 = np.histogram(n_feats_imp, bins=bins)

plt.plot(y[1][1:]+0.5,y[0],c='black', label = 'Sleap')
plt.plot(y2[1][1:]+0.5,y2[0],c='blue', label = 'Sleap + imputation')
    
#plt.semilogy()
plt.xlim(1,7)
plt.ylim(bottom=0)
plt.legend(fontsize=14)
plt.title("animal id: "+ str(animal_id))
plt.ylabel("# frames")
# 
plt.show()

(2069710,)


In [100]:
feature_ids = np.array([0,5,6,7,8,9])
d = np.load('/media/cat/1TB/dan/cohort1/slp/animalID_0_alldata.npy')[:,feature_ids]
d_imp = np.load('/media/cat/1TB/dan/cohort1/slp/animalID_0_alldata_imputed.npy')


idx_all = np.arange(d.shape[0])
ctr=0
sizes = (np.arange(1,7,1)*0.1)[::-1]

fig = plt.figure(figsize=(10,10))
while ctr<10:
#while True:
    idx = np.random.choice(idx_all, 1)[0]
    idx2 = np.where(np.isnan(d[idx][:,0])==False)[0]
    idx3 = np.where(np.isnan(d_imp[idx,:,0])==False)[0]

    if idx2.shape[0] != idx3.shape[0] and idx2.shape[0]>=5:

    #if idx2.shape[0]>=2:

        #idx3 = np.where(np.isnan(d_imp[idx,:,0])==False)[0]

        print (idx, idx2, idx3, sizes[idx2])

        print ("preimupted: ", d[idx])

        plt.subplot(2,5,ctr+1)
        ss = np.zeros((6))+np.nan
        ss[idx3] = sizes[idx3]
        plt.scatter(d_imp[idx,:,0],
                    d_imp[idx,:,1],
                    s=ss,
                    c='blue')

        ss = np.zeros((6))+np.nan
        ss[idx2] = sizes[idx2]
        print ("sizes: ", ss)
        plt.scatter(d[idx,:,0],
                    d[idx,:,1],
                    s=ss,
                    c='black')
 
        ctr+=1
        plt.title(str(idx)+ " " +str(idx2), fontsize=3, pad=0.8)
        plt.xticks([])
        plt.yticks([])
        plt.xlim(0,1280)
        plt.ylim(0,1280)

if True:
    plt.savefig('/home/cat/impute.png', dpi=300)
    plt.close()
else:
    plt.show()


[[[ 815.5662    248.4631  ]
  [ 828.0727    212.18002 ]
  [ 856.01      204.01614 ]
  [ 865.5865    185.22838 ]
  [ 864.8135    163.06822 ]
  [ 858.07983   144.82709 ]]

 [[ 815.54205   248.4631  ]
  [ 828.0727    212.19121 ]
  [ 855.9853    204.01614 ]
  [ 865.57605   185.23795 ]
  [ 864.8342    163.0905  ]
  [ 858.12427   144.85498 ]]

 [[ 815.54205   248.4631  ]
  [ 828.0727    212.24358 ]
  [ 855.9771    204.01614 ]
  [ 865.5792    185.2284  ]
  [ 864.85626   163.09923 ]
  [ 858.176     144.87238 ]]

 ...

 [[1003.80945   123.85146 ]
  [ 976.056     120.63238 ]
  [ 943.31604   132.44987 ]
  [ 920.0462    124.01467 ]
  [ 904.6494    111.888695]
  [ 893.4489    107.69545 ]]

 [[ 940.41614    99.940254]
  [ 915.248     123.777016]
  [ 907.865     144.44054 ]
  [ 886.83026   155.86105 ]
  [ 867.07166   164.44434 ]
  [ 858.3316    174.203   ]]

 [[ 939.6487     99.35089 ]
  [ 915.185     123.777016]
  [ 907.865     144.2303  ]
  [ 887.1728    156.26631 ]
  [ 867.86      165.56894 ]
  [ 

In [ ]:
#########################################
#### VISUALIZE IMPUTATIONS + MOVIES #####
#########################################




In [12]:
from sys import getsizeof
data = np.load('/media/cat/1TB/dan/cohort1/slp/2020_3_16_12_57_12_418305_compressed.npy')
print (getsizeof(data), data.shape, data.nbytes)

80630288 (89989, 4, 14, 2) 80630144


In [13]:
data = np.load('/media/cat/1TB/dan/cohort1/slp/centre_rotated_animalID_0_0_1.npz', allow_pickle=True)


feats = data['feats_rotated']
print (feats.shape)

(2069710, 6, 2)


In [ ]:
################################    
#### UNSHIFT/UNROTATE RESULTS ##
################################

# visualize before and after results in video form


In [7]:
data = np.load('/media/cat/1TB/dan/cohort1/slp/2020_3_16_01_57_27_327194_compressed_median_filtered_outliers_centre_aligned_0_1.npz',
              allow_pickle=True)

feats = data['features_full']
print (feats.shape)

(89988, 4, 6, 2)


In [38]:
f1 = 0
f2 = 1
feats = load_processed_data_stand_alone(f1,f2,I.March16_file_order, I.root_dir, I.animal_ids, data_type=1, remove_nans=False)
print (np.vstack(feats[0]).shape)
for k in range(I.animal_ids.shape[0]):
    np.save(I.root_dir+'/animalID_'+str(k)+'_alldata.npy', np.vstack(feats[k]))



   note: pipeline currently applied only to Cohort1 March 16th datasets
 DATA SIZES: 
 female [n_samples, n_featres, xy]:  (2069710, 14, 2)
 male:                               (2069710, 14, 2)
 pup1:                               (2069710, 14, 2)
 pup2:                               (2069710, 14, 2)
(2069710, 14, 2)


In [21]:
idx = np.unique(np.where(np.isnan(feats)==False)[0])
print (idx, idx.shape)
feats2 = feats[idx]
print (feats2.shape)
print (feats2)


[ 5856  9187  9188 ... 60156 60157 60158] (9581,)
(9581, 4, 6, 2)
[[[[ 0.0000000e+00  0.0000000e+00]
   [-2.6053378e-16 -2.7597610e+01]
   [ 4.1872730e+00 -4.4353676e+01]
   [ 1.4419242e+01 -5.2421986e+01]
   [ 2.2772881e+01 -7.9629517e+01]
   [ 1.8599001e+01 -8.9770401e+01]]

  [[           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]]

  [[           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]]

  [[           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]
   [           nan            nan]]]


 [[[ 0.0000000e+00  0.0000000e+00]
   [ 5.1549951e-15 -3.3143169e+01]
   [ 1.9924431

In [ ]:

# # load filtered + outliner triaged data with Nans removed
# I.cb.load_processed_data(f1, f2,
#                          data_type=0,
#                          remove_nans=True)
        
        
# I.load_models()


# # test against ground truth/cleand data 
# I.fname_dropout = '/home/cat/feats_dropout.tsv'
# res, idx_drop = I.predict_imputation_ground_truth(drops=np.arange(3,6,1)) # drops = 'fixed' or None

# # check missing features in the data
# # I.calculate_missing_features()

# # 
# # I.plot_imputation_results(features_array, animal_id, idx_test, res, idx_drop)
# plt.suptitle("animal "+str(animal_id)+ " imputed vs. ground truth",fontsize=20)

# # 
# I.evaluate_imputation_error(features_array, animal_id, res, idx_train, idx_test)
# plt.suptitle("animal "+str(animal_id)+ "  Egocentric (fixed nose) errors (pixels)",fontsize=20)
# #plt.suptitle("animal "+str(animal_id)+ " NON-Egocentric (fixed nose) errors (pixels)",fontsize=20)


# plt.show()

In [45]:
# select some random drops for each
drops = np.load('/home/cat/temp_f1f2_drops.npy')
print (drops)

idx1 = np.where(drops==1)[0]
idx2 = np.where(drops==2)[0]
print (idx1.shape)
print (idx2.shape)

idx3 = np.intersect1d(idx1, idx2)
print (idx3.shape)
print (drops[idx3])



[[1 5 4]
 [0 5 1]
 [2 4 0]
 ...
 [0 4 1]
 [4 5 0]
 [4 1 5]]
(109565,)
(109445,)
(44043,)
[[5 1 2]
 [1 2 0]
 [4 1 2]
 ...
 [1 5 2]
 [5 2 1]
 [1 4 2]]


In [31]:
idx = np.random.sample(np.arange(6), 1)
print (idx)




TypeError: random_sample() takes at most 1 positional argument (2 given)

In [13]:
# compute the error vs. ground truth
I.evaluate_imputation_multivariate()


# VAE CODE NOT USED NOW 
# make_vae_data()

# df1 = evaluate_imputation_vae()




KeyboardInterrupt: 

In [61]:
# 
dropped_features= [0,1,2]
I.plot_vae_scatter(dropped_features)  
    
#        
I.plot_multiple_imputation_results()


(218641, 12)
(1093205, 12)
(218641, 12)
(218641, 12)
(1093205, 12)
(218641, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)
match_drop: [0 1 2 3 4 5]
(5, 12)


In [ ]:
##########################################################
##########################################################
##########################################################
from scipy.spatial import cKDTree
import joblib

def knn_triage(th, pca_wf):
    tree = cKDTree(pca_wf)
    dist, ind = tree.query(pca_wf, k=6)
    dist = np.sum(dist, 1)
    idx_keep1 = dist <= np.percentile(dist, th)
    return idx_keep1



# Fit the PCA object, but do not transform the data
for k in range(4):
    ax=plt.subplot(2,2,k+1)
    
    temp = features_array[k]
    d = []
    clrs = []
    for p in range(len(temp)):
        d.append(temp[p])
        clrs.extend(np.zeros(temp[p].shape[0])+p)
    
    clrs = np.array(clrs)
    d = np.vstack(d)
    print ("D: ", d.shape)
    d = d.reshape(d.shape[0],-1)
    continue
    #d = sklearn.preprocessing.normalize(d)

    # remove 1% of outliers
    if True:
        th = 95  # % of data to keep
        idx_keep = knn_triage(th, d)
        print (" d before traige: ", d.shape)
        d = d[idx_keep]
        print (" d after traige: ", d.shape)
        clrs = clrs[idx_keep]
    
    
    if False:
        pca = PCA(2)

        print ("... data into pca: ", d.shape)

        feats_pca = pca.fit_transform(d)
        print (feats_pca.shape)

        # 
        plt.scatter(feats_pca[::5,0],
           feats_pca[::5,1],
            #c=np.arange(feats_pca.shape[0])[::5],
            c=clrs[::5],
            alpha=.05)
        
    if True:
        
#         import gpumap
#         #from sklearn.datasets import load_digits

#         #digits = load_digits()
#         print ("Data into gpumap: ", d.shape)
#         feats_pca = gpumap.GPUMAP().fit_transform(d)
#         print ("Data out of gpumap: ", feats_pca.shape)

        import umap
    
        umap = umap.UMAP(n_components=2,
                        init='random',
                        random_state=0)
        
        d = d[::2]
        clrs = clrs[::2]
        
        print ("... data into umap: ", d.shape)
        
        if False:
            umap_ = umap.fit(d) #[::10])
            feats_pca = umap_.transform(d)
        else:
            feats_pca = umap.fit_transform(d) #[::10])
        
        
            # remove 1% of outliers
        if True:
            th = 90  # % of data to keep
            idx_keep = knn_triage(th, feats_pca)
            print (" d before traige: ", feats_pca.shape)
            feats_pca = feats_pca[idx_keep]
            print (" d after traige: ", feats_pca.shape)
            clrs = clrs[idx_keep]
        
        plt.scatter(feats_pca[:,0],
               feats_pca[:,1],
                #c=np.arange(feats_pca.shape[0])[::5],
                c=clrs,
                alpha=.05)
    if False:
        
        #from openTSNE import TSNE
        #print ("... data into tsne: ", d.shape)
        #feats_pca = TSNE().fit(d)
        
        
        from fastTSNE import TSNE

        tsne = TSNE(
            n_components=2, perplexity=30, learning_rate=100, early_exaggeration=12,
            n_jobs=4, 
            #angle=0.5, 
            initialization='random', metric='euclidean',
            n_iter=750, early_exaggeration_iter=250, neighbors='exact',
            negative_gradient_method='bh', min_num_intervals=10,
            #ints_in_inverval=2, 
            #late_exaggeration_iter=100, 
            #late_exaggeration=4,
        )
        
        # 
        feats_pca = tsne.fit(d)

        print (" output: ", feats_pca.shape)


        plt.scatter(feats_pca[:,0],
            feats_pca[:,1],
            #c=np.arange(feats_pca.shape[0])[::5],
            c=clrs,
            alpha=.05)

    # 
    plt.title("Animal:"+str(k))
    
    
plt.suptitle("Static vertically aligned postures",fontsize=20)
plt.show()
#plt.show()

In [15]:
lens = [218641, 94647, 132861, 176982]

lens = np.array(lens)
print (np.round(lens/(23*89900)*100,2))

[10.57  4.58  6.43  8.56]


In [44]:
#################################################
############### IMPUTE MISSING DATA #############
#################################################
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit([[1, 2], [3, 6], [4, 8], [np.nan, 3], [7, np.nan]])
X_test = [[np.nan, 2], [6, np.nan], [np.nan, 6]]
# the model learns that the second feature is double the first

print(np.round(imp.transform(X_test)))

[[ 1.  2.]
 [ 6. 12.]
 [ 3.  6.]]


In [15]:
data = np.load('/home/cat/angles_ego_animalID_3_duration_25.npy')
print (data.shape)

idx = np.where(data<0)[0]
idx2 = np.unique(idx)
print (idx2.shape)
print (data)




(177945, 25)
(77827,)
[[0.         0.         0.         ... 0.15860032 0.15860032 0.15860032]
 [0.         0.         0.         ... 0.15860032 0.15860032 0.15860032]
 [0.         0.         0.         ... 0.15860032 0.15860032 0.15860032]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
0 [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.16658191 0.15625526 0.15625526
 0.15625526 0.15625526 0.15625526 0.15625526 0.15625526 0.15625526
 0.15625526 0.15625526 0.16658191 0.14309867 0.15860032 0.15860032
 0.15860032]

1 [0.         0.         0.         0.         0.         0.
 0.         0.         0.16658191 0.15625526 0.15625526 0.15625526
 0.15625526 0.15625526 0.15625526 0.15625526 0.15625526 0.15625526
 0.15625526 0.16658191 0.14309867 0.15860032 0.15860032 0.15860

 0.69592977]

261 [0.         0.         0.         0.         0.         0.
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977]

262 [0.         0.         0.         0.         0.         0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977]

263 [0.         0.         0.         0.         0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977
 0.69592977]

264 [0.         0.         0.         0.69592977 0.69592977 0.69592977
 0.69592977 0.69592977 0.69592977 0.69592977 0.69592977 0.6959297


860 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

861 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

862 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

863 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

864 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

865 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

866 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

867 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

868 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

869 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

870 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

871 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]

872

In [ ]:
##############################################
########## FEATURIZE BEHAVIOR CHUNKS #########
##############################################
from sklearn import decomposition
import sklearn

fig = plt.figure()
X_all = []
n_events = []
for animal_id in animal_ids:
    X = X4[animal_id].copy()
    X = X.reshape(X.shape[0], -1)
    print (X.shape)
    X_all.append(X)
    n_events.append(X.shape[0])

#     
X_all = np.vstack(X_all)
print (X_all.shape)
X = sklearn.preprocessing.normalize(X_all)

#
if True:
    pca = decomposition.PCA(n_components=3)

    X_pca = pca.fit_transform(X_all)
    print (X_pca.shape)
    
if False:
    import umap
    umap = umap.UMAP(n_components=2,
                    init='random',
                    random_state=0)

    umap_ = umap.fit(X_all[::10])

    X_pca = umap_.transform(X_all)
        

print ("plotting: ", X_pca.shape)


print (n_events)
fig=plt.figure()
for k in range(4):
    ax = plt.subplot(2,2,k+1)
    start = np.int32(n_events[:k]).sum()
    end = np.int32(n_events[:k+1]).sum()
    print (start, end)
    plt.scatter(X_pca[start:end,0],
                X_pca[start:end,1],
               alpha=.1)

plt.show()